In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, trim, lower

spark = SparkSession.builder \
    .appName("Assignments") \
    .master("local[*]") \
    .getOrCreate()


In [2]:
cons = spark.read.parquet("path/to/consumerInternet.parquet")
start = spark.read.option("header", True).csv("path/to/startups.csv")
combined = cons.unionByName(start)
combined.createOrReplaceTempView("all_data")


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/admin/path/to/consumerInternet.parquet.

In [4]:
startups = spark.read.option("header", True).csv("C:/Users/admin/Desktop/spark_assignment/startup.csv")
consumer = spark.read.parquet("C:/Users/admin/Desktop/spark_assignment/consumerInternet.parquet")

combined = consumer.unionByName(startups)
combined.createOrReplaceTempView("startups")


In [5]:
spark.sql("SELECT COUNT(*) AS pune_startups FROM startups WHERE City = 'Pune'").show()


+-------------+
|pune_startups|
+-------------+
|          105|
+-------------+



In [6]:
spark.sql("""
SELECT COUNT(*) AS seed_angel_pune 
FROM startups 
WHERE City = 'Pune' AND InvestmentnType IN ('Seed Funding', 'Angel Funding')
""").show()


+---------------+
|seed_angel_pune|
+---------------+
|             52|
+---------------+



In [7]:
from pyspark.sql.functions import regexp_replace

combined = combined.withColumn("clean_amount", regexp_replace("Amount_in_USD", ",", ""))
combined = combined.withColumn("clean_amount", combined["clean_amount"].cast("double"))
combined.createOrReplaceTempView("startups")

spark.sql("""
SELECT SUM(clean_amount) AS total_pune_funding
FROM startups WHERE City = 'Pune'
""").show()


+------------------+
|total_pune_funding|
+------------------+
|         6.33082E8|
+------------------+



In [8]:
spark.sql("""
SELECT Industry_Vertical, COUNT(*) AS count
FROM startups
GROUP BY Industry_Vertical
ORDER BY count DESC
LIMIT 5
""").show()


+-----------------+-----+
|Industry_Vertical|count|
+-----------------+-----+
|Consumer Internet|  941|
|       Technology|  478|
|        eCommerce|  186|
|              nan|  171|
|       Healthcare|   70|
+-----------------+-----+



In [9]:
from pyspark.sql.functions import year, to_date, col, row_number
from pyspark.sql.window import Window

df = combined.withColumn("year", year(to_date("Date", "dd/MM/yyyy")))
df.createOrReplaceTempView("startups")

top_investors = spark.sql("""
    SELECT year, Investors_Name, SUM(clean_amount) AS total
    FROM startups
    GROUP BY year, Investors_Name
""")

windowSpec = Window.partitionBy("year").orderBy(col("total").desc())
ranked = top_investors.withColumn("rank", row_number().over(windowSpec)).filter("rank = 1")
ranked.select("year", "Investors_Name", "total").show()


Py4JJavaError: An error occurred while calling o91.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 14.0 failed 1 times, most recent failure: Lost task 1.0 in stage 14.0 (TID 15) (DESKTOP-33JDV5N executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '01/07/015' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDif

In [10]:
combined.select("Date").show(10, truncate=False)


+----------+
|Date      |
+----------+
|01/09/2018|
|03/09/2018|
|04/09/2018|
|05/09/2018|
|06/09/2018|
|11/09/2018|
|02/08/2018|
|03/08/2018|
|07/08/2018|
|07/08/2018|
+----------+
only showing top 10 rows



In [11]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

top_investors = df.groupBy("year", "Investors_Name").sum("clean_amount").withColumnRenamed("sum(clean_amount)", "total")

windowSpec = Window.partitionBy("year").orderBy(col("total").desc())
ranked = top_investors.withColumn("rank", row_number().over(windowSpec)).filter("rank = 1")

ranked.select("year", "Investors_Name", "total").show()


Py4JJavaError: An error occurred while calling o127.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 16.0 failed 1 times, most recent failure: Lost task 1.0 in stage 16.0 (TID 18) (DESKTOP-33JDV5N executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '01/07/015' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDi

In [12]:
from pyspark.sql.functions import to_date, year, col

df = combined.withColumn("ParsedDate", to_date("Date", "dd/MM/yyyy"))
df = df.withColumn("year", year(col("ParsedDate")))
df.select("Date", "ParsedDate", "year").show(5)


+----------+----------+----+
|      Date|ParsedDate|year|
+----------+----------+----+
|01/09/2018|2018-09-01|2018|
|03/09/2018|2018-09-03|2018|
|04/09/2018|2018-09-04|2018|
|05/09/2018|2018-09-05|2018|
|06/09/2018|2018-09-06|2018|
+----------+----------+----+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("clean_amount", regexp_replace("Amount_in_USD", ",", "").cast("double"))
df.select("Amount_in_USD", "clean_amount").show(10)


+-------------+------------+
|Amount_in_USD|clean_amount|
+-------------+------------+
|  3,50,00,000|       3.5E7|
|    63,90,000|   6390000.0|
|          N/A|        null|
| 15,72,00,000|     1.572E8|
| 10,00,00,000|       1.0E8|
|    55,00,000|   5500000.0|
|    70,00,000|   7000000.0|
|     1,20,000|    120000.0|
|    10,00,000|   1000000.0|
|  3,50,00,000|       3.5E7|
+-------------+------------+
only showing top 10 rows



In [14]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

top_investors = df.groupBy("year", "Investors_Name").sum("clean_amount").withColumnRenamed("sum(clean_amount)", "total")

windowSpec = Window.partitionBy("year").orderBy(col("total").desc())
ranked = top_investors.withColumn("rank", row_number().over(windowSpec)).filter("rank = 1")

ranked.select("year", "Investors_Name", "total").show()


Py4JJavaError: An error occurred while calling o182.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 19.0 failed 1 times, most recent failure: Lost task 1.0 in stage 19.0 (TID 22) (DESKTOP-33JDV5N executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '01/07/015' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDi

In [15]:
from pyspark.sql.functions import row_number, col
from pyspark.sql.window import Window

top_investors = df.groupBy("year", "Investors_Name")\
    .sum("clean_amount")\
    .withColumnRenamed("sum(clean_amount)", "total")\
    .filter("total is not null")

windowSpec = Window.partitionBy("year").orderBy(col("total").desc())
ranked = top_investors.withColumn("rank", row_number().over(windowSpec)).filter("rank = 1")

ranked.show(10, truncate=False)


Py4JJavaError: An error occurred while calling o209.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 20.0 failed 1 times, most recent failure: Lost task 1.0 in stage 20.0 (TID 24) (DESKTOP-33JDV5N executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '01/07/015' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDi

In [16]:
from pyspark.sql.functions import col, length

# Only accept rows with date strings of length 10 and year >= 2000
valid_df = df.filter(
    (length("Date") == 10) &
    (col("Date").substr(-4, 4).cast("int") >= 2000)
)


In [17]:
from pyspark.sql.functions import substring

# Add year column
df_with_year = valid_df.withColumn("year", substring(col("Date"), -4, 4).cast("int"))


In [18]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Group and rank
top_investors = df_with_year.groupBy("year", "Investors_Name")\
    .sum("clean_amount")\
    .withColumnRenamed("sum(clean_amount)", "total")\
    .filter("total is not null")

windowSpec = Window.partitionBy("year").orderBy(col("total").desc())
ranked = top_investors.withColumn("rank", row_number().over(windowSpec)).filter("rank = 1")

# Show results
ranked.select("year", "Investors_Name", "total").show(10, truncate=False)


+----+----------------------------------------+------------+
|year|Investors_Name                          |total       |
+----+----------------------------------------+------------+
|2015|Steadview Capital and existing investors|7.0E8       |
|2016|Verlinvest, DSG Consumer Partners       |NaN         |
|2017|Softbank                                |2.5E9       |
|2018|N/A                                     |6.09085951E8|
|2019|Westbridge Capital                      |3.9E9       |
|2020|Tiger Global Management                 |2.0E8       |
+----+----------------------------------------+------------+



In [19]:
from pyspark.sql.functions import row_number, col
from pyspark.sql.window import Window

# Assume df has clean_amount column & City
win_city = Window.partitionBy("City").orderBy(col("clean_amount").desc())

top_startup_city = df.withColumn("rk", row_number().over(win_city)) \
                     .filter("rk = 1") \
                     .select("City", "Startup_Name", "clean_amount")

top_startup_city.show(truncate=False)


+----------------------+-----------------+------------+
|City                  |Startup_Name     |clean_amount|
+----------------------+-----------------+------------+
|null                  |Mswipe           |2.5E7       |
|Agra                  |Shoekonnect      |null        |
|Ahemadabad            |Infibeam         |6230000.0   |
|Ahemdabad             |Foodmemories     |null        |
|Ahmedabad             |LendingKart      |3.2E7       |
|Amritsar              |eBikeGo          |300000.0    |
|Andheri               |Indus OS         |5750000.0   |
|Bangalore             |Flipkart         |2.5E9       |
|Bangalore / Palo Alto |Streamoid        |1000000.0   |
|Bangalore / SFO       |ZenDrive         |1.35E7      |
|Bangalore / San Mateo |Cloud Lending    |8000000.0   |
|Bangalore / USA       |Infoworks        |5000000.0   |
|Bangalore/ Bangkok    |Zilingo          |8000000.0   |
|Belgaum               |SenseGiz         |500000.0    |
|Bengaluru             |Rapido Bike Taxi |3.9E9 

In [20]:
# Count startups per SubVertical per Year
counts = df.groupBy("year", "SubVertical").count()

# Use window to compute difference between consecutive years
from pyspark.sql.functions import lag
from pyspark.sql.window import Window

win = Window.partitionBy("SubVertical").orderBy("year")
growth = counts.withColumn("prev_count", lag("count").over(win)) \
               .withColumn("growth", col("count") - col("prev_count"))

# Find max growth value
max_growth = growth.groupBy("SubVertical") \
                   .agg({"growth": "max"}) \
                   .withColumnRenamed("max(growth)", "max_growth") \
                   .orderBy(col("max_growth").desc())

max_growth.show(10, truncate=False)


Py4JJavaError: An error occurred while calling o308.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 30.0 failed 1 times, most recent failure: Lost task 1.0 in stage 30.0 (TID 33) (DESKTOP-33JDV5N executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '01/07/015' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDi

In [21]:
df.filter("year IS NULL OR SubVertical IS NULL").show()


Py4JJavaError: An error occurred while calling o313.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 35) (DESKTOP-33JDV5N executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '01/07/015' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDi

In [22]:
from pyspark.sql.functions import col

# Remove rows with incorrectly formatted years (length != 4)
df_valid = df.filter("length(SUBSTRING_INDEX(Date, '/', -1)) = 4")


In [23]:
from pyspark.sql.functions import substring_index

df_with_year = df_valid.withColumn("year", substring_index("Date", "/", -1))


In [24]:
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window

# Count startups per year per SubVertical
subv_counts = df_with_year.groupBy("year", "SubVertical").count()

# Define window
win = Window.partitionBy("SubVertical").orderBy("year")

# Calculate growth
growth = subv_counts.withColumn("prev_count", lag("count").over(win)) \
                    .withColumn("growth", col("count") - col("prev_count"))

# Max growth per SubVertical
max_growth = growth.groupBy("SubVertical") \
                   .agg({"growth": "max"}) \
                   .withColumnRenamed("max(growth)", "max_growth") \
                   .orderBy(col("max_growth").desc())

max_growth.show(10, truncate=False)


+-----------------------------+----------+
|SubVertical                  |max_growth|
+-----------------------------+----------+
|Online Lending Platform      |7         |
|Online Education Platform    |3         |
|Online lending platform      |3         |
|Online Food Delivery         |2         |
|Online Pharmacy              |2         |
|B2B Marketplace              |1         |
|Fitness Mobile App           |1         |
|Data Analytics platform      |1         |
|ECommerce Marketplace        |1         |
|Non-Banking Financial Company|1         |
+-----------------------------+----------+
only showing top 10 rows

